### ***Изменение формы выражения текста без потери смысла***

Цель: Изменять структуру предложений и лексический состав текста, сохраняя его первоначальное значение.
Задания:
- Реализуйте методы перестановки порядка слов, замены синонимами и сокращения избыточных выражений, проверяя сохранение общего смысла.Метрика оценки: Человеческая экспертиза (оценка читабельности и сохранения смысла), сравнение показателей COMET или CHRF++.


In [ ]:
!pip install transformers sacrebleu torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
from transformers import pipeline
from sacrebleu.metrics import CHRF
import random

In [ ]:

paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws", tokenizer="Vamsi/T5_Paraphrase_Paws")

simplifier = pipeline("text2text-generation", model="tuner007/pegasus_paraphrase", tokenizer="tuner007/pegasus_paraphrase")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def generate_paraphrases(text, num_return_sequences=3):
    input_text = f"paraphrase: {text} </s>"
    results = paraphraser(
        input_text,
        max_length=128,
        num_beams=5,
        num_return_sequences=num_return_sequences,
        temperature=1.5
    )
    return [r["generated_text"] for r in results]

def evaluate_with_chrf(original, augmented_list):
    metric = CHRF()
    scores = []
    for aug in augmented_list:
        score = metric.sentence_score(aug, [original])
        scores.append(score.score)
        print(f"\n Аугментированный текст:\n{aug}\n CHRF++: {score.score:.2f}")
    return scores


In [ ]:
def simplify_text(text):
        output = simplifier(text, max_length=60, do_sample=False)
        return output[0]['generated_text']
        return text

In [ ]:
if __name__ == "__main__":
    sample = "He was able to work at the present time, but due to the fact that he was ill, he stayed at home."

    # Генерируем перефразированные варианты
    paraphrased = generate_paraphrases(sample, num_return_sequences=3)

    # Упрощаем оригинальный текст
    simplified = simplify_text(paraphrased[0])

    # Собираем все варианты
    all_augmented = paraphrased + [simplified]

    # Вывод и оценка
    print(f"📌 Оригинал:\n{sample}")
    evaluate_with_chrf(sample, all_augmented)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


📌 Оригинал:
He was able to work at the present time, but due to the fact that he was ill, he stayed at home.

 Аугментированный текст:
He was able to work at the moment, but due to the fact that he was ill he stayed at home .
 CHRF++: 79.53

 Аугментированный текст:
He was able to work at the present time, but due to the fact that he was ill he stayed at home .
 CHRF++: 95.38

 Аугментированный текст:
He was able to work at the moment, but due to the fact that he was ill he stayed at home.
 CHRF++: 79.53

 Аугментированный текст:
He stayed at home because he was ill and was able to work at the moment.
 CHRF++: 50.72


### ***Автономное маскирование личных данных в текстах***

Цель: Автоматизировать процесс удаления чувствительной личной информации из текстовых источников.
Задания:
- Запрограммируйте решение для автоматического выявления и замещения имен собственных, адресов электронной почты, номеров телефонов и другой персональной информации.Метрика оценки: Точность и полнота выявления чувствительных данных, отсутствие ложноположительных срабатываний.
- Опишите и реализуйте продвинутые стратегии маскировки данных с использованием генераторов синтетических данных (Data Generation Techniques).Модели сравнения: Fake Name Generator, DataFaker
- Настройте нейросетевую модель на основе Transformer для автоматизации процесса пересказа текста различными способами.Модели сравнения: Simple Paraphrase Engine, Quillbot paraphraser

In [ ]:
!pip install presidio_analyzer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.9 MB/s eta 0:00:00


In [ ]:
!pip install presidio_anonymizer

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.1 MB/s eta 0:00:00


In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from faker import Faker
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import re
import random
import difflib

In [ ]:
# Инициализация инструментов
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()
faker = Faker()

# Продвинутая модель парафраза
model_name = "ramsrigouthamg/t5_paraphraser"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ===== 1. Обнаружение и маскирование персональных данных =====
def mask_with_presidio(text):
    results = analyzer.analyze(text=text, language="en")
    anonymized_result = anonymizer.anonymize(text=text, analyzer_results=results)
    return anonymized_result.text, results

def mask_with_regex(text):
    patterns = {
        'EMAIL': r"[\w\.-]+@[\w\.-]+",
        'PHONE': r"\+?\d[\d\-\(\) ]{7,}\d",
        'NAME': r"\b([A-Z][a-z]+\s[A-Z][a-z]+)\b",
        'ADDRESS': r"\d+\s+\w+\s+(Street|St|Avenue|Ave|Road|Rd|Blvd|Lane|Ln)"
    }
    masked_text = text
    for label, pattern in patterns.items():
        masked_text = re.sub(pattern, f"<{label}>", masked_text)
    return masked_text

# ===== 2. Синтетические данные с Faker =====
def generate_synthetic_profile():
    return {
        "name": faker.name(),
        "email": faker.email(),
        "phone_number": faker.phone_number(),
        "address": faker.address().replace("\n", ", ")
    }

# ===== 3. Парафраз текста =====
def paraphrase_text(text, num_return_sequences=3):
    text = remove_data_in_brackets(text)
    input_text = f"paraphrase: {text} </s>"
    encoding = tokenizer.encode_plus(input_text, padding='max_length', return_tensors="pt", max_length=128, truncation=True)
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=128,
        do_sample=True,
        top_k=120,
        top_p=0.95,
        early_stopping=True,
        num_return_sequences=num_return_sequences
    )
    return [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]

def remove_data_in_brackets(text):
    """Removes data enclosed in <> brackets from a given text.

    Args:
        text: The input text string.

    Returns:
        The text with data in <> brackets removed.
    """
    return re.sub(r'<.*?>', '', text)


# ===== 5. Сравнение парафразов =====
def compare_paraphrases(original, paraphrased_list):
    print("\nСравнение парафразов:")
    for i, para in enumerate(paraphrased_list):
        similarity = difflib.SequenceMatcher(None, original, para).ratio()
        print(f"{i+1}. {para} (Сходство: {similarity:.2f})")


In [ ]:
def fill_presidio_with_synthetic(masked_text, entities):
    profile = generate_synthetic_profile()

    replacements = {
        "PERSON": profile["name"],
        "EMAIL_ADDRESS": profile["email"],
        "PHONE_NUMBER": profile["phone_number"],
        "LOCATION": profile["address"].split(",")[0]
    }

    for entity in entities:
        label = entity.entity_type
        if f"<{label}>" in masked_text and label in replacements:
            masked_text = masked_text.replace(f"<{label}>", replacements[label], 1)

    return masked_text, profile


In [ ]:
if __name__ == "__main__":
    original_text = """
        Maria Ivanovna will not be able to go to work as she is ill and remains at home at 123 Main Street, Springfield. But you can email her at john.doe@gmail.com or call +1-202-555-0182.
    """

    print("=== Оригинальный текст ===")
    print(original_text)

    # Маскирование Presidio
    presidio_masked, presidio_entities = mask_with_presidio(original_text)
    print("\n=== Presidio Замаскированный ===")
    print(presidio_masked)

    # Маскирование Regex
    regex_masked = mask_with_regex(original_text)
    print("\n=== Regex Замаскированный ===")
    print(regex_masked)

    # Замена на синтетические данные
    synthetic_text, synthetic_profile = fill_presidio_with_synthetic( presidio_masked, presidio_entities)
    print("\n=== Синтетические данные (Faker) ===")
    print(synthetic_text)

    paraphrases = paraphrase_text(presidio_masked)
    compare_paraphrases(original_text, paraphrases)


=== Оригинальный текст ===

        Maria Ivanovna will not be able to go to work as she is ill and remains at home at 123 Main Street, Springfield. But you can email her at john.doe@gmail.com or call +1-202-555-0182.
    

=== Presidio Замаскированный ===

        <PERSON> will not be able to go to work as she is ill and remains at home at <LOCATION>, <LOCATION>. But you can email her at <EMAIL_ADDRESS> or call <PHONE_NUMBER>.
    

=== Regex Замаскированный ===

        <NAME> will not be able to go to work as she is ill and remains at home at 123 <NAME>, Springfield. But you can email her at <EMAIL> or call <PHONE>.
    

=== Синтетические данные (Faker) ===

        Greg Watson will not be able to go to work as she is ill and remains at home at 844 Blake Mews Apt. 398, 844 Blake Mews Apt. 398. But you can email her at rhondagreen@example.net or call 213-592-4901.
    

Сравнение парафразов:
1. Will not be able to go to work as she is ill and remains at home. But you can email her o